In [1]:
# Cell 1: Imports and Setup
import os, gc, warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.linear_model import Ridge
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
import xgboost as xgb

warnings.filterwarnings("ignore")
np.random.seed(42)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:10]:
        print(os.path.join(dirname, filename))

/kaggle/input/eee-g513/train.csv
/kaggle/input/eee-g513/test.csv
/kaggle/input/eee-g513/cost_of_living.csv


In [2]:
# Cell 2: Load Data
DATA_PATH = "/kaggle/input/eee-g513"

train_raw = pd.read_csv(f"{DATA_PATH}/train.csv")
test_raw = pd.read_csv(f"{DATA_PATH}/test.csv")
colv = pd.read_csv(f"{DATA_PATH}/cost_of_living.csv")

print(f"Train: {train_raw.shape}, Test: {test_raw.shape}, COL: {colv.shape}")
print(f"\nTarget stats:\n{train_raw['salary_average'].describe()}")

Train: (6525, 6), Test: (2790, 5), COL: (1528, 56)

Target stats:
count      6480.000000
mean      53169.866584
std       27592.877914
min        3836.556547
25%       28697.390996
50%       54673.064063
75%       72742.000000
max      157747.634644
Name: salary_average, dtype: float64


In [3]:
# Cell 3: Simple COL Merge - Just median per city
keys = ['city_id'] if 'city_id' in train_raw.columns else ['country', 'state', 'city']

# Aggregate COL with median
colv_agg = colv.groupby(keys, as_index=False).median(numeric_only=True)

train = train_raw.merge(colv_agg, on=keys, how='left', suffixes=('', '_drop'))
test = test_raw.merge(colv_agg, on=keys, how='left', suffixes=('', '_drop'))

train = train[[c for c in train.columns if not c.endswith('_drop')]]
test = test[[c for c in test.columns if not c.endswith('_drop')]]

print(f"After merge - Train: {train.shape}, Test: {test.shape}")

After merge - Train: (6525, 59), Test: (2790, 58)


In [4]:
# Cell 4: Target Cleaning
TARGET = "salary_average"

before = len(train)
train = train[train[TARGET].notna() & (train[TARGET] > 0)].reset_index(drop=True)
print(f"Dropped {before - len(train)} invalid targets")

feat_cols = [c for c in train.columns if c in test.columns and c != TARGET]
print(f"Features: {len(feat_cols)}")

Dropped 45 invalid targets
Features: 58


In [5]:
# Cell 5: Global Feature Engineering (computed once on full train)
def create_global_features(train_df, test_df):
    """
    Create features using FULL train data (computed once).
    This matches your simpler approach that scored 0.24.
    """
    # Compute encodings on FULL train
    encodings = {}
    
    # Country-role
    encodings['country_role'] = train_df.groupby(['country', 'role'])['salary_average'].agg(['mean', 'median', 'std']).to_dict()
    
    # Country
    encodings['country'] = train_df.groupby('country')['salary_average'].agg(['mean', 'median', 'std', 'count']).to_dict()
    
    # State
    encodings['state'] = train_df.groupby('state')['salary_average'].agg(['mean', 'median', 'count']).to_dict()
    
    # Role
    encodings['role'] = train_df.groupby('role')['salary_average'].agg(['mean', 'median', 'std', 'min', 'max']).to_dict()
    
    # Global fallbacks
    global_mean = train_df['salary_average'].mean()
    global_median = train_df['salary_average'].median()
    global_std = train_df['salary_average'].std()
    
    # Apply to both train and test
    for df in [train_df, test_df]:
        # Country-role
        df['te_country_role_mean'] = df.apply(
            lambda x: encodings['country_role']['mean'].get((x['country'], x['role']), global_mean), axis=1)
        df['te_country_role_median'] = df.apply(
            lambda x: encodings['country_role']['median'].get((x['country'], x['role']), global_median), axis=1)
        
        # Country
        df['te_country_mean'] = df['country'].map(encodings['country']['mean']).fillna(global_mean)
        df['te_country_median'] = df['country'].map(encodings['country']['median']).fillna(global_median)
        df['te_country_std'] = df['country'].map(encodings['country']['std']).fillna(global_std)
        
        # State
        df['te_state_mean'] = df['state'].map(encodings['state']['mean']).fillna(df['te_country_mean'])
        df['te_state_median'] = df['state'].map(encodings['state']['median']).fillna(df['te_country_median'])
        
        # Role
        df['te_role_mean'] = df['role'].map(encodings['role']['mean']).fillna(global_mean)
        df['te_role_median'] = df['role'].map(encodings['role']['median']).fillna(global_median)
        df['te_role_std'] = df['role'].map(encodings['role']['std']).fillna(global_std)
        df['te_role_range'] = df['role'].map(encodings['role']['max']).fillna(global_mean) - \
                              df['role'].map(encodings['role']['min']).fillna(global_mean)
        
        # COL interactions
        col_cols = [c for c in df.columns if 'col_' in c.lower() or 'rent' in c.lower() or 'grocer' in c.lower()]
        if len(col_cols) > 0:
            for col in col_cols:
                df[col] = df[col].fillna(df[col].median())
            
            df['col_mean'] = df[col_cols].mean(axis=1)
            df['col_median'] = df[col_cols].median(axis=1)
            df['col_std'] = df[col_cols].std(axis=1).fillna(0)
            
            # Adjusted features
            df['te_role_per_col'] = df['te_role_mean'] / (df['col_mean'] + 1)
            df['te_country_per_col'] = df['te_country_mean'] / (df['col_mean'] + 1)
            df['te_role_x_col'] = df['te_role_mean'] * df['col_mean']
        
        # Ratios
        df['country_vs_role'] = df['te_country_mean'] / (df['te_role_mean'] + 1)
        df['state_vs_country'] = df['te_state_mean'] / (df['te_country_mean'] + 1)
        
        # Log transforms
        df['log_te_role_mean'] = np.log1p(df['te_role_mean'])
        df['log_te_country_mean'] = np.log1p(df['te_country_mean'])
        
    return train_df, test_df

train, test = create_global_features(train, test)
feat_cols = [c for c in train.columns if c in test.columns and c != TARGET]
print(f"Features after engineering: {len(feat_cols)}")

Features after engineering: 79


In [6]:
# Cell 6: Minimal Preprocessing
num_cols = [c for c in feat_cols if pd.api.types.is_numeric_dtype(train[c])]
cat_cols = [c for c in feat_cols if c not in num_cols]

print(f"Numeric: {len(num_cols)}, Categorical: {len(cat_cols)}")

# Only clip extreme outliers (0.5% and 99.5%)
for c in num_cols:
    q_low, q_high = train[c].quantile([0.005, 0.995])
    train[c] = train[c].clip(q_low, q_high)
    test[c] = test[c].clip(q_low, q_high)

# Impute with median
for c in num_cols:
    med = train[c].median()
    train[c] = train[c].fillna(med)
    test[c] = test[c].fillna(med)

Numeric: 75, Categorical: 4


In [7]:
# Cell 7: CV Setup
group_key = 'city_id' if 'city_id' in train.columns else 'city'
groups = train[group_key] if group_key in train.columns else train['country']

gkf = GroupKFold(n_splits=5)

def rmspe(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = (y_true > eps) & np.isfinite(y_true) & np.isfinite(y_pred)
    if mask.sum() == 0:
        return np.inf
    return np.sqrt(np.mean(((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2))

print(f"CV Strategy: GroupKFold on '{group_key}' with {gkf.get_n_splits()} folds")

CV Strategy: GroupKFold on 'city_id' with 5 folds


In [8]:
# Cell 8: LightGBM - Focus on single best model
lgb_oof = np.zeros(len(train))
lgb_test = np.zeros(len(test))
y_oof_true = np.zeros(len(train))

lgb_params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'num_leaves': 63,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.5,
    'max_depth': 10,
    'verbosity': -1,
    'seed': 42
}

print("\n=== LightGBM Training ===")
for fold, (tr_idx, va_idx) in enumerate(gkf.split(train, groups=groups), 1):
    X_tr = train.iloc[tr_idx][feat_cols]
    y_tr = train.iloc[tr_idx][TARGET].values
    X_va = train.iloc[va_idx][feat_cols]
    y_va = train.iloc[va_idx][TARGET].values
    
    cat_features = [c for c in feat_cols if not pd.api.types.is_numeric_dtype(train[c])]
    for c in cat_features:
        X_tr[c] = X_tr[c].astype('category')
        X_va[c] = X_va[c].astype('category')
    
    dtrain = lgb.Dataset(X_tr, label=np.log1p(y_tr), categorical_feature=cat_features)
    dvalid = lgb.Dataset(X_va, label=np.log1p(y_va), categorical_feature=cat_features)
    
    model = lgb.train(
        lgb_params,
        dtrain,
        valid_sets=[dvalid],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0)
        ]
    )
    
    X_te = test[feat_cols].copy()
    for c in cat_features:
        X_te[c] = X_te[c].astype('category')
    
    va_pred = np.expm1(model.predict(X_va, num_iteration=model.best_iteration))
    te_pred = np.expm1(model.predict(X_te, num_iteration=model.best_iteration))
    
    lgb_oof[va_idx] = va_pred
    lgb_test += te_pred / gkf.n_splits
    y_oof_true[va_idx] = y_va
    
    fold_score = rmspe(y_va, va_pred)
    print(f"Fold {fold}: RMSPE = {fold_score:.4f} | Iterations = {model.best_iteration}")

lgb_cv_score = rmspe(y_oof_true, lgb_oof)
print(f"\nLGBM OOF RMSPE: {lgb_cv_score:.4f}")


=== LightGBM Training ===
Fold 1: RMSPE = 0.0556 | Iterations = 771
Fold 2: RMSPE = 0.0354 | Iterations = 910
Fold 3: RMSPE = 0.0459 | Iterations = 693
Fold 4: RMSPE = 0.0449 | Iterations = 936
Fold 5: RMSPE = 0.0452 | Iterations = 1997

LGBM OOF RMSPE: 0.0471


In [9]:
# Cell 9: Create Submission
id_col = 'ID' if 'ID' in test_raw.columns else 'id'
test_ids = test_raw[id_col].values if id_col in test_raw.columns else np.arange(1, len(test_raw) + 1)

submission = pd.DataFrame({
    'ID': test_ids,
    'salary_average': lgb_test
})

print(f"\nSubmission Validation:")
print(f"  Shape: {submission.shape}")
print(f"  Expected: ({len(test_raw)}, 2)")
print(f"  Min: ${submission['salary_average'].min():,.2f}")
print(f"  Max: ${submission['salary_average'].max():,.2f}")
print(f"  Mean: ${submission['salary_average'].mean():,.2f}")

submission.to_csv('submission.csv', index=False)
print("\n✓ Submission saved!")
print(submission.head(10))


Submission Validation:
  Shape: (2790, 2)
  Expected: (2790, 2)
  Min: $4,944.92
  Max: $124,139.83
  Mean: $55,137.47

✓ Submission saved!
   ID  salary_average
0   1    80168.761167
1   2    84155.107174
2   3    86535.116387
3   4    65220.089803
4   5    54671.250753
5   6    67741.196871
6   7    54706.187518
7   8    66385.627188
8   9    85561.458516
9  10    87102.786358


In [10]:
# Cell 10: Cleanup
gc.collect()
print("\n✓ Pipeline complete!")
print(f"OOF RMSPE: {lgb_cv_score:.4f}")


✓ Pipeline complete!
OOF RMSPE: 0.0471
